**ToF signal data preparation**

Our ToF data comes in a CSV file where the first column is time in seconds and the second column is voltage in volts. Also, the first 4 rows of the CSV are text headers so we will need to skip over them. For this code to be super easy to use, I suggest having the ToF files in a Google drive. All the repositary stuff with Git confuses me so much(!!!), probably to a trained coder it would be very easy to adapt this to import a CSV file from elsewhere.



In [77]:
 import numpy as np #importing modules to be used
 import pandas as pd
 import matplotlib

 from google.colab import drive
drive.mount('/content/drive') #this authorises the user to access files in their Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To get the file path of the CSV, have the CSV in Google drive, then click the folder icon on the LHS of the screen in Colab. You'll get a list of all your folders - find the CSV you want, right click and select 'Copy filepath'.

In [78]:
tof_data = "/content/drive/My Drive/ToF stuff/C18_2_II_p1_h26v00000.csv" #file path of CSV goes here
df_ToF = pd.read_csv(tof_data, skiprows = 5) #creates a dataframe of the data
df_ToF.columns = ['t', 'V'] #assigns column titles 
ToF_array = df_ToF.values #puts dataframe into an array

df_ToF # raw data in nice format!

,t,V
0,-0.000049,0.002284
1,-0.000049,0.002271
2,-0.000048,0.002206
3,-0.000048,0.002232
4,-0.000047,0.002245
...,...,...
995,0.000448,0.004784
996,0.000449,0.004641
997,0.000449,0.004758
998,0.000450,0.004758


So now we have created an array with all our data in it, using Pandas. Next we want to convert the time values from seconds into microseconds, and the raw voltage into micro amps. To do this we will define a function that has an input of the raw data array and the load resistor, and outputs a dataframe of t in $\mu$s and i in $\mu$A.

NB: A weakness of the ToF_prep function defined below is that the calculation of dark current is NOT failsafe. I just estimated a range for points for dark current that were consistent with my results but could totally change based on what timebase you use. However, if the same timebase is used for all data on a particular device, it's usually safe to use the same range for all biases. The split( ) function could probably utilised somewhere in here but I don't got the brain power to figure it out right now.

In [104]:
def ToF_prep(ToF_array, R_L): 

  t_us = [] # t in us
  i_uA = [] # i in uA
  i_P_uA = []  # photocurrent in uA

  for x in ToF_array: 
    t = x[0]*10e6 # seconds to micro seconds
    t_us.append(t)
  
    i = (x[1]*10e6)/R_L # volts to micro amps
    i_uA.append(i)

  i_D = sum(i_uA[5:91]) / 85 # be super careful about this range, if using large time base range could be smaller

  for y in i_uA: # takes dark current away from raw current
    i_P = y - i_D
    i_P_uA.append(i_P)

  #print(t_us) # these potentially helpful to make sure you're getting what you expect!
  #print(i_uA)
  #print(i_D)
  #print(i_P_uA)

  prepared_df = pd.DataFrame(t_us, i_P_uA) # puts lists of values into new dataframe
  prepared_df.reset_index(inplace=True)
  prepared_df.columns = ['photocurrent', 'time']

  return prepared_df

ToF_prep(ToF_array, 2130)



,photocurrent,time
0,-1.039527,-495.0
1,-1.100653,-490.0
2,-1.406287,-485.0
3,-1.284034,-480.0
4,-1.222907,-475.0
...,...,...
995,10.697563,4480.0
996,10.025121,4485.0
997,10.575309,4490.0
998,10.575309,4495.0


Now we just need to write the dataframe to a CSV and export it to Google drive. 

In [103]:
ready_to_export = ToF_prep(ToF_array, 2130)


ready_to_export.to_csv(r'/content/drive/My Drive/ToF stuff/prepared_tof_data.csv', index = False)

Congrats u have just prepared your ToF data. I'm too thick to work out how to find the time of arrival in Python so good luck comrade.